In [4]:
import time
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import os
from PIL import Image
from scipy import ndimage


%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
# Step 1: Load and preprocess the CIFAR-10 dataset

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#try-catch system for file errors

import os 
def read_file(file):
    try:
        dataset=open('/Users/ryanvitalez/Desktop/cifar-10-batches-py/data_batch_1', 'r')
        return dataset
    except OSError as e:
        message ="OS Error: {0}".format(e)
        return message
    
file = '/Users/ryanvitalez/Desktop/cifar-10-batches-py/data_batch_1'
output = read_file(file)

#print (output)

# Step 1.1: Load and preprocess the dataset

dataset_folder = '/Users/ryanvitalez/Desktop/cifar-10-batches-py'
data_dict = {}

# Step 1.2: Iterate through each batch file in the folder

for file in os.listdir(dataset_folder):
    if file.startswith("data_batch"):
        batch_path = os.path.join(dataset_folder, file)
        batch_dict = unpickle(batch_path)
        
        # Merge the data from each batch into a single dictionary
        if not data_dict:
            data_dict = batch_dict
        else:
            for key in data_dict:
                data_dict[key] += batch_dict[key]

train_images = data_dict[b'data']
train_labels = data_dict[b'labels']

# Step 1.3: Normalize the pixel values

train_images = train_images / 255.0

# Step 1.4: Split the dataset into training and validation sets

val_split = 0.2
num_samples = train_images.shape[0]
num_val_samples = int(num_samples * val_split)

val_images = train_images[:num_val_samples]
val_labels = train_labels[:num_val_samples]
train_images = train_images[num_val_samples:]
train_labels = train_labels[num_val_samples:]


no problem


In [8]:
# since dnn_app_utils_v2 do not work for me:

def sigmoid(Z):
    
    """
    Implements the sigmoid activation in numpy
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    cache -- return Z as well, useful during back propagation
    """
    
    A = 1/(1 + np.exp(-Z))
    cache = Z
    
    return A, cache

In [9]:
def relu(Z):
    
    """
    Implement the ReLU function.
    
    Arguments:
    Z -- output of the linear layer, of any shape
    
    Returns: 
    A -- post-activation parameter, of the same shape as Z
    cache -- a python dictionary containing "A" ; stored for computing the backward pass efficiently
    
    """
    
    A = np.maximum(0, Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z
    return A, cache


In [10]:
def relu_backward(dA, cache):
    
    """
    Implement the backward propagation for a single ReLU unit.
    
    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently
    
    Returns:
    dZ -- gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy = True)
    
    # when <= =, set dZ to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert(dZ.shape == Z.shape)
    
    return dZ


In [11]:
def sigmoid_backward(dA, cache):
    
    """
    Implement the backward propagation for a single SIGMOID unit
    
    Arugments: 
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently
    
    Returns:
    dZ -- gradient of the cost with respect to Z
    
    """
    
    Z = cache
    
    s == 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    
    assert(dZ.shape == Z.shape)
    
    return dZ


In [80]:
def load_data():
    
    train_dataset = h5py.File('/Users/ryanvitalez/Desktop/cifar-10-batches-py/data_batch_1', 'r')
    print ('a')
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
    print ('b')
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels
    print ('c')
    
    test_dataset = h5py.File('/Users/ryanvitalez/Desktop/cifar-10-batches-py/test_batch', 'r')
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels
    
    classes = np.array(test_dataset["list_classes"][:]) # list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [14]:
def initialize_parameters(n_x, n_h, n_y):
    
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    Parameters -- python dictionary containing the following parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(a)
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameter

In [15]:
def initialize_parameters_deep(layer_dims):
    
    """
    Argument:
    layer_dims -- python array (list) containing the dimenstions of each layer in the network
    
    Returns:
    Parameters -- python dictionary containing the parameters "W1", "b1", ... , "WL", "bL":
                    Wl -- wight matrix of shape (layer_dims[l], layer_dims[l - 1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) / np.sqrt(layer_dims[l - 1]) 
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters

In [16]:
def linear_forward(A, W, b):
    
    """
    Implement the linear part of a layer's forward propagation
    
    Arguments:
    A -- activation from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix; numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector; numpy array of shape (size of the current layer, 1)
    
    Returns:
    Z -- the input of the activation function, also call pre-activation parameter
    cache -- a python dictionary containing "A", "W", and "b" ; stored for computing the backward pass efficiently
    """
    
    Z = W.dot(A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache == (A, W, b)
    
    return Z, cache


In [17]:
def linear_activation_forward(A_prev, W, b, activation):
    
    """
    Implement the forward propagation for the LINEAR -> ACTIVATION layer
    
    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation function to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    A -- the output of the activation function, also called the post-activation value
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             store for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z) 
        
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [18]:
def L_model_forward(X, parameters):
    
    """
    Implement forward propagation for the (LINEAR -> RELU) * (L-1) - > LINEAR -> SIGMOID computation
    
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_dee()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """
    
    caches = []
    A = X
    L = len(parameters) // 2
    
    # Implement [LINEAR -> RELU] * (L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    # Implement [LINEAR -> SIGMOID] * (L-1). Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A, parameters['W' + str(l)], parameters['b' + str(l)], activation = "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1, X.shape[1]))
    
    return AL, caches

In [19]:
def compute_cost(AL, Y):
    
    """
    Implement the cost function
    
    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)
    
    Returns: 
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]
    
    cost = (1./m) * (-np.dot(Y, np.log(AL).T) - np.dot(1 - Y, np.log(1 - AL).T))
    
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    return cost

In [20]:
def linear_backward(dZ, cache):
    
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1./m * np.dot(dZ, A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T, dZ)
    
    assert(dA_prev.shape == A.shape)
    assert(db.shape == b.shape)
    assert(dW.shape == W.shape)
    
    return dA_prev, dW, db

In [21]:
def linear_activation_backward(dA, cache, activation):
    
    """
    Implement the backward propagation for the LINEAR -> ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db

In [23]:
def L_model_backward(AL, Y, caches):
    
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (there are (L-1) or them, indexes from 0 to L-2)
                the cache of linear_activation_forward() with "sigmoid" (there is one, index L-1)
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    
    grads = {} 
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L - 1)):
        current_cache = cache[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backwards(grads["dA" + str(l + 2)], current_cache, activation = "relu")
        
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
    return grads
        
    

In [24]:
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 
    
    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
        
    return parameters
    

In [37]:
def predict(X, y, parameters):
    """
    This function is used to predict the results of a  L-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    n = len(parameters) // 2
    p = np.zeros((1, m))
    
    probas, caches = L_model_forward(X, parameters)
    
    for i in range(0, probas.shape[1]):
        if probas[0, i] > 0.5:
            p[0, i] = 1
        else:
            p[0, 1] = 0
            
    print("Accuracy: " + str(np.sum((p == y)/m)))
    
    return p

In [54]:

def print_mislabeled_images(dataset_folder, X, classes, y, p):
    """
    Plots images where predictions and truth were different.
    X -- dataset
    y -- true labels
    p -- predictions
    """
    
    X = {dataset_folder}
    a = p + y
    mislabeled_indices = np.asaaray(np.where(a == 1))
    plt.rcParams['figure.figsize'] = (40.0, 40.0) 
    num_images = len(mislabeled_indices[0])
    for i in range(num_images):
        index = mislabeled_indices[1][i]
        
        plt.subplot(2, num_images, i + 1)
        plt.imshow(X[index].reshape(64, 64, 3). interpolation == 'nearest')
        plt.axis('off')
        plt.title("Prediction: " + classes[int(p[0, index])].decode("utf-8") + "\n Class:" + classes[y[0, index]].decode("utf-8"))
                  

In [81]:

train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = load_data()

OSError: Unable to open file (file signature not found)